# 第五章 在未 token 的数据上进行预训练

**本章介绍**：

- 计算训练集和验证集的损失，以评估训练过程中 LLM 生成文本的质量
- 实现训练函数并对 LLM 进行预训练
- 保存和加载模型权重，以便继续训练 LLM
- 加载 OpenAI 的预训练权重

在前几章中，我们实现了数据采样、注意力机制，并编写了LLM 架构的代码。本章我们将主要关注如何实现训练函数并对LLM 进行预训练，如图 5.1 所示。

**图 5.1 对于构建 LLM 的三个主要阶段的心智模型，包括在通用文本数据集上预训练 LLM，以及在 token 的数据集上对其进行微调。本章将主要关注 LLM 的预训练，包括实现训练代码，评估性能，以及保存和加载模型权重。**

![fig5.1](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-1.jpg?raw=true)

如图 5.1 所示，我们将进一步学习基本的模型评估技术，以便衡量生成文本的质量，这是在训练过程中优化LLM的关键步骤。此外，我们还将探讨如何加载预训练的权重，这将为我们在后续章节中对LLM进行微调提供坚实的基础。

**权重参数（Weight parameters）**

在 LLM 和其他深度学习模型的背景下，权重（*weights*）指的是学习过程中需要调整的可训练参数。这些权重也被称为权重参数（*weight parameters*）或 简称为参数（*parameters*）。在像 PyTorch 这样的框架中，这些权重存储在线性层中，例如我们在第 3 章中实现多头注意力模块和第 4 章中的 GPTModel 时使用的。在初始化一个层（`new_layer = torch.nn.Linear(...)`）后，我们可以通过` .weight `属性访问其权重，即`new_layer.weight`。此外，为了方便，PyTorch允 许直接访问模型的所有可训练参数，包括权重和偏置，通过 `model.parameters()` 方法，我们将在后面实现模型训练时使用。

## 5.1 评估文本生成模型

我们将从上一章的代码出发，介绍如何使用 LLM 进行文本生成，然后讨论评估生成文本质量的基本方法。本节以及本章剩余部分的内容概述如图 5.2 所示。

**图 5.2 本章的主题内容如下。我们首先回顾上一章的文本生成内容，然后实现在预训练阶段进行模型评估的基本技术。**

![fig5.2](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-2.jpg?raw=true)

如图 5.2 所示，下一小节我们将回顾上一章末尾设置的文本生成内容，然后在后续的小节中深入研究文本评估和计算训练及验证损失。

### 5.1.1 使用 GPT 生成文本

在本节，我们将初始化 LLM，并简要回顾在第四章中实现的文本生成过程。首先，我们将初始化一个 GPT 模型，该模型将在本章中被评估和训练。我们将使用第四章中的 GPTModel 类和 GPT_CONFIG_124M 字典来完成模型的初始化：

In [21]:
import torch
from chapter04 import GPTModel
GPT_CONFIG_124M = {
"vocab_size": 50257,
"context_length": 256, #A
"emb_dim": 768,
"n_heads": 12,
"n_layers": 12,
"drop_rate": 0.1, #B
"qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttent

对于 GPT_CONFIG_124M 字典，我们与上一章相比做出的唯一调整是将上下文长度（context_length）缩减到 256 个 tokens。这种改变降低了模型训练的计算压力，使得在普通的笔记本电脑上进行训练成为可能。

拥有 1.24 亿参数的 GPT-2 模型原本被配置为处理 1024 个 tokns。在训练过程结束后，我们将在本章末尾更新上下文大小设置，并加载预训练的权重，以便与配置为 1024 tokens 上下文长度的模型一起工作。

借助 GPTmodel 实例，我们采用了上一章介绍的 `generate_text_simple `函数，并引入了两个实用的函数，`text_to_token_ids` 和 `token_ids_to_text`。这些函数方便我们在文本和 token 表示之间进行转换，我们将在本章中频繁使用它们。为了提供更清晰的理解，我们在深入代码之前，通过图 5.3 来展示这个过程。

**图 5.3 文本生成过程包括将文本编码为 token ID，随后被 LLM 处理为 logit 向量。之后这些 logit 向量被转换回 token ID，最后再被解码为文本形式。**

![fig5.3](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-3.png?raw=true)

图 5.3 描绘了使用 GPT 模型进行的文本生成的三步。首先，如第二章所述，分词器将输入文本转换为一系列的 token ID。其次，模型接收这些 token ID，并生成相应的 logit，这些 logit 是向量，代表词汇表中每个令牌的概率分布，如第四章所述。最后，这些 logit 被转换回 token ID，分词器将其解码为人类可读的文本，从而完成从文本输入到文本输出的循环。

我们实现了如下文本生成过程的代码：

In [22]:
import tiktoken
from chapter04 import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # 添加批次维度
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # 删除批次维度
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


使用前面的代码，模型会生成以下文本：

```
Output text:
Every effort moves you rentingetic wasnم refres RexMeCHicular stren
```

从输出结果来看，模型显然还无法生成连贯的文本，因为它还未经过训练。为了定义何为"连贯的"或"高质量的"文本，我们需要实现一种数值化的方法来评估生成的内容。这种方法将使我们能够在整个训练过程中监控并提升模型的性能。

接下来的部分将介绍我们如何为生成的输出计算损失指标（*loss metric*）。这个损失作为训练进度的衡量和成功的标志。此外，在后续关于微调 LLM 的章节中，我们将回顾评估模型质量的其他方法。

### 5.1.2 计算文本生成损失

在本节中，我们将深入探讨一种通过计算文本生成损失，以量化评估训练过程中生成的文本质量的技术。我们将通过一个实际的例子，逐步深入解析这个主题，以便让概念更加清晰并易于实践。我们先简短回顾下第二章的数据加载和第四章的`generate_text_simple`函数生成文本。

图 5.4 以五步流程清晰地描绘了从输入文本到 LLM 生成文本的整个过程。

**图 5.4 对于图片左侧显示的三个输入，我们会为每一个输入 token 计算一个向量，该向量包含对应于词汇表中每个 token 的概率分数。每个向量中概率分数最高的索引位置代表最可能的下一个 token ID。选择与最高概率分数相关联的这些 token ID，并将其映射回一个文本，这个文本就代表模型生成的文本。**

![fig5.4](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-4.jpg?raw=true)

图 5.4 中的文本生成流程详细描述了第四章中 `generate_text_simple` 函数的内部工作原理。在本节后面计算生成文本质量的损失之前，我们需要先执行这些相同的初始步骤。

图 5.4 以一个只有 7 个 toekns 的小型词汇表为例，概述了文本生成过程，以便在单页上展示此图像。然而，我们的 GPT 模型使用的是一个包含 50,257 个单词的大型词汇表；因此，在接下来的代码中， token  ID 的范围将是 0 到 50,256，而不仅仅是 0 到 6。

此外，为了简化，图 5.4 仅展示了一个文本示例（"every effort moves"）。在接下来实现图 5.4 步骤的代码示例中，我们将使用两个输入示例（"every effort moves" 和 "I really like"）作为 GPT 模型的输入。

考虑两个输入示例，这些示例已经被转换为对应的 token ID，对应于图 5.4 中的步骤 1：

In [23]:
inputs = torch.tensor([[16833, 3626, 6100], # ["every effort moves",
                       [40, 1107, 588]]) # "I really like"]

"目标（`targets`）"包含我们希望模型生成的生成对应输入的 token ID：
Matching these inputs, the `targets` contain the token IDs we aim for the model to produce:

In [24]:
targets = torch.tensor([[3626, 6100, 345 ], # [" effort moves you",
                        [588, 428, 11311]]) # " really like chocolate"]

请注意，目标与输入相同，只不过是向前移动了一个位置，这是我们在第二章实现数据加载器时所讨论过的概念。这种移位策略对于训练模型预测序列中的下一个元素至关重要。

我们将输入送入模型以计算两个输入示例的逻辑向量，每个示例由三个 toekns组成，并应用 softmax 函数将这些逻辑值转换为概率分数，这对应于图 5.4 中的步骤 2：

In [25]:
with torch.no_grad(): #A
    logits = model(inputs)
probas = torch.softmax(logits, dim=-1) # 词表中每个 token 的概率
print(probas.shape)

torch.Size([2, 3, 50257])


由此得出的概率分数（probas）张量维度如下：

`torch.Size([2, 3, 50257])`

第一个数字 2，代表输入中的两个示例（行），也被称为批量大小。第二个数字 3，代表每个输入（行）中的 token 数量。最后一个数字则对应于嵌入的维度，这是由词表大小决定的，正如我们在前面的章节中讨论的。

在通过 softmax 函数将逻辑值转换为概率后，使用我们在第四章中实现的 `generate_text_simple` 函数将这些概率分数再次转换为文本，如图 5.4 的步骤 3-5 所示。

我们可以通过对概率分数应用 argmax 函数来实现步骤 3 和步骤 4，从而获得相应的 token ID：

In [26]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


考虑到我们有两个输入批次，每个批次都包含 3 个 token，将 argmax 函数应用于概率分数（如图 5.4 的步骤 3 所示）会产生两组输出，每组都包含3个预测的 token ID：

```
Token IDs:
tensor([[[16657], # First batch
        [ 339],
        [42826]],
       [[49906], # Second batch
        [29669],
        [41751]]])
```

最后，第 5 步将 token ID 转换回文本：

In [27]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix


当我们解码这些 token 时，我们发现这些输出 token 与我们希望模型生成的目标 token 完全不同：

```
Targets batch 1: effort moves you
Outputs batch 1: Armed heNetflix
```

模型产生的随机文本与目标文本不同，这是因为它还没有经过训练。现在，我们将通过一种被称为“损失”的方式，对模型生成的文本性能进行数值化评估，如图5.4所示。这种方法不仅对于衡量生成文本的质量有重要作用，同时也是实现后续训练函数的基础。我们将利用这个函数来更新模型的权重，从而提升生成文本的质量。

**图 5.5 在本节的剩余部分，我们将实现文本评估函数。在接下来的一节中，我们将把这个评估函数应用到用于模型训练的整个数据集上。**

![fig5.5](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-5.png?raw=true)

在本节剩余部分中，我们将实现文本评估过程的部分，如图5.5所示。这个过程是为了衡量生成的 token 与正确预测（目标）之间的“距离”。在本章后续的训练函数中，我们将利用这些信息来调整模型权重，以便生成的文本更接近（理想情况下与）目标文本。

模型训练的目标是提升正确目标 token ID 对应索引位置的 softmax 概率，如图 5.6 所示。这个 softmax 概率也被应用于我们在本节后续部分要实现的评估指标中，用于对模型生成的输出进行数值评估：正确位置的概率越高，效果就越好。

**图 5.6 未进过训练时，模型随机生成下一个 token 的概率向量。模型训练的目标是最大化目标 token ID 对应的概率值。**

![fig5.6](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-6.jpg?raw=true)

请注意，图 5.6 展示了一个只有 7 个 token 的紧凑词汇表的 softmax 概率，以便将所有信息都整合到一个图形中。这意味着初始的随机值将大约在 1/7（约等于 0.14）附近。

不过，我们在 GPT-2 模型中使用的词汇有 50,257 个 tokens，因此大多数初始概率都会在 0.00002（1/50,257 ）附近。

对于两个输入文本中的每一个，我们可以通过以下代码打印与目标 token 对应的初始 softmax 概率分数：

In [28]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)
text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Text 1: tensor([7.4541e-05, 3.1061e-05, 1.1563e-05])
Text 2: tensor([3.9836e-05, 1.6783e-05, 4.7559e-06])


每个批次的 3 个目标 token ID 概率如下：

```
Text 1: tensor([7.4541e-05, 3.1061e-05, 1.1563e-05])
Text 2: tensor([3.9836e-05, 1.6783e-05, 4.7559e-06])
```

 LLM 的训练目标是最大化这些概率值，尽可能使它们接近1。这样，模型在生成下一个 token 时，将始终选择目标 token —— 即句子中的下一个词。

**反向传播（Backpropagation）**

我们如何才能最大化目标 tokens 对应的 softmax 概率值呢？总的来说，我们会更新模型的权重，使得模型对我们希望生成的各个 token ID 输出更高的值。权重的更新是通过一种名为反向传播的过程来完成的，这是训练深度神经网络的标准技术（关于反向传播和模型训练的更多详细信息，请参见附录 A 的 A.3 至 A.7 节）。

反向传播需要一个损失函数，该函数用于计算模型预测输出（在这里，是目标 token ID 对应的概率）与实际期望输出之间的差距。这个损失函数用于衡量模型预测结果与目标值之间的偏离程度。

在本节剩余部分，我们将计算两个示例批次，即 `target_probas_1` 和 `target_probas_2` 的概率分数的损失。主要步骤已在图 5.7 中展示。

**图 5.7 计算损失包含多个步骤。步骤 1 至 3 用于计算目标张量相对应的 token 概率。然后，在步骤 4 至 6 中，这些概率经过对数转换并取平均。**

![fig5.7](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-7.jpg?raw=true)

由于我们已经按照图 5.7 中的步骤 1-3 计算出了 `target_probas_1` 和 `target_probas_2`，接下来我们将进行步骤 4，对这些概率分数应用对数函数。

In [29]:
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([ -9.5042, -10.3796, -11.3677, -10.1307, -10.9951, -12.2561])


结果如下：

```
tensor([ -9.5042, -10.3796, -11.3677, -10.1308, -10.9951, -12.2561])
```

在数学优化过程中，处理概率得分的对数比直接处理得分本身更为便捷。这个主题超出了本书的讨论范围，但我在一次讲座中对此进行了详细阐述，你可以在附录B的参考资料部分找到相关链接。

接下来，我们通过计算平均值将这些对数概率合并为一个分数（图 5.7 中的步骤 5）：

In [30]:
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

tensor(-10.7722)


得出的平均对数概率得分如下：

`tensor(-10.7722)`

我们的目标是通过在训练过程中更新模型的权重，使平均对数概率尽可能接近 0，这部分我们将在 5.2 节中实现。

然而，在深度学习中，常见的做法并不是提升平均对数概率至 0，而是降低负平均对数概率至 0。负平均对数概率即平均对数概率乘以 -1，这对应于图 5.7 中的第 6 步：

In [31]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(10.7722)


这打印了张量`(10.7722)`

这个负值（-10.7722 变成 10.7722）在深度学习中被称为交叉熵损失（*cross entropy*）。

PyTorch 在这里派上了用场，因为它已经内置了一个 `cross_entropy` 函数，可以为我们处理图 5.7 中的所有这 6 个步骤。

**交叉熵损失（Cross entropy loss）**

交叉熵损失在机器学习和深度学习中是一种常用的度量方法，用于衡量两个概率分布之间的差异——通常是标签的真实分布（在这里，是数据集中的 token）和模型的预测分布（例如，由 LLM 生成的 token 概率）。

在机器学习领域，尤其是在像 PyTorch 这样的框架中，`cross_entropy` 函数用于计算离散结果的度量值，这与目标 token 在模型生成的 token 概率下的负平均对数概率类似。因此，交叉熵和负平均对数概率这两个术语在实践中经常被互换使用。

在应用交叉熵函数之前，让我们简单回顾一下 logits 和目标张量的形状：

In [32]:
print("Logits shape:", logits.shape)
print("Targets shape:", targets.shape)

Logits shape: torch.Size([2, 3, 50257])
Targets shape: torch.Size([2, 3])


形状如下：

```
Logits shape: torch.Size([2, 3, 50257])
Targets shape: torch.Size([2, 3])
```

如我们所见，logits 张量有三个维度：批量大小、token 数量和词汇表大小。而 targets 张量有两个维度：批量大小和 token 数量。

对于 PyTorch 中的交叉熵损失函数，我们希望通过在批次维度上合并这些张量来扁平化这些张量：

In [33]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()
print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])


得到的张量维度如下：

```
Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])
```

请注意，目标值是我们期望 LLM 生成的 token ID，而 logits 则包含了模型在经过 softmax 函数获取概率分数之前的未经缩放的输出值。

在之前，我们应用了 softmax 函数，选择了与目标 ID 对应的概率分数，并计算了负平均对数概率。PyTorch 的 `cross_entropy` 函数将为我们处理所有这些步骤：

In [34]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(10.7722)


产生的损失与我们之前手动实现图 5.7 中所示的各个步骤时获得的损失相同：

```
tensor(10.7722)
```

**困惑度（Perplexity）**

困惑度是一种常用的评估指标，经常与交叉熵损失一起用于评价如 LLM 这类任务的模型性能。它提供了一种更易于理解的方式，帮助我们理解模型在预测序列中下一个 token 时的不确定性。

困惑度衡量的是模型预测的概率分布与数据集中实际单词分布的匹配程度。与损失类似，较低的困惑度表明模型的预测更接近实际的分布。

困惑度可以通过公式 `perplexity = torch.exp(loss)` 来计算。当我们将这个公式应用到之前计算的损失值时，得到的结果是 `tensor(47678.8633)`。

困惑度通常被认为比原始损失值更易于理解，因为它代表了模型在每一步中对有效词汇量的不确定性。在这个例子中，这意味着模型在词汇表中的 47,678 个单词或 token 中，不确定哪一个会被生成为下一个 token。

在本节中，我们计算了两个小文本输入的损失，以作说明。下一节，我们将对整个训练集和验证集进行损失计算。

### 5.1.3 计算训练集和验证集损失

在本节中，我们首先准备了将在本章后面用于训练 LLM 的训练集和验证集。接着，我们计算了训练集和验证集的交叉熵，如图 5.8 所示，这是模型训练过程中的重要组成部分。

**图 5.8 在前一节中，我们已经计算了交叉熵损失，现在我们将这种损失计算方法应用到我们即将用于模型训练的整个文本数据集上。**

![fig5.8](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-8.jpg?raw=true)

为了计算图 5.8 中展示的训练集和验证集的损失，我们使用了一个非常小的文本数据集——Edith Wharton 的短篇故事 "The Verdict"，这是我们在第二章已经使用过的数据集。选择公共领域的文本，我们避开了任何与使用权相关的问题。此外，我们选择这样一个小数据集的原因是，它允许我们在标准的笔记本电脑上，即使没有高端的 GPU，也能在几分钟内执行代码示例，这对于教学目的来说非常有利。

对于对此感兴趣的读者，您也可以利用本书提供的附加代码，准备一个更大规模的数据集。这个数据集由 Project Gutenberg 的超过 60,000 本公共领域图书组成，并可以在这些数据上训练一个 LLM （详细内容请参见附录 D）。

**预训练 LLM 的成本**

为了更好地理解我们项目的规模，我们可以参考一下训练拥有 70 亿参数的 Llama 2 模型，这是一个相对知名且公开可用的 LLM。这个模型在昂贵的 A100 GPU 上运行了 184,320 个小时，处理了 2 万亿个 token。在撰写本文时，AWS 上运行一个 8xA100 云服务器的费用大约为每小时 30 美元。粗略估计，这样一个 LLM 的总训练成本大约为 690,000 美元（计算方式为 184,320 小时除以 8，然后乘以 30 美元）。

下面的代码将加载我们在第 2 章中使用过的短篇故事 "The Verdict"：

In [35]:
file_path = "the-verdict.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_data = file.read()

加载数据集后，我们可以检查数据集中的字符数和 token 数：

In [36]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 20479
Tokens: 5145


输出如下：

```
Characters: 20479
Tokens: 5145
```

尽管这段文本只有 5145 个 token，对于训练一个大型 LLM 来说，可能显得太少了。然而，正如我们之前提到的，这是出于教学目的，使我们能够在几分钟而非几周的时间内运行代码。此外，我们将在本章的最后，将 OpenAI 的预训练权重加载到我们的 GPTModel 代码中。


接下来，我们将数据集分为训练集和验证集，并利用第二章中的数据加载器来为 LLM 训练准备批次数据。这个过程在图 5.9 中进行了直观的展示。

**图 5.9 在准备数据加载器的过程中，我们首先将输入文本分割为训练集和验证集。接着，我们对文本进行 token 化处理（为了简化，这里仅展示了训练集部分的处理过程），并将 token 化后的文本划分为用户指定长度的块（在此例中为 6）。最后，我们打乱各行的顺序，并将划分后的文本组织成批次（在此例中，批次大小为 2），这样我们就可以用它们进行模型训练了。**

![fig5.9](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-9.jpg?raw=true)

为了便于可视化，我们在图 5.9 中将最大长度设定为 6，这主要是由于空间限制。然而，在我们实际实现的数据加载器中，我们将最大长度设定为 LLM 所支持的 256 个 token 的上下文长度，这样可以让 LLM 在训练过程中接触到更长的文本。

**使用可变长度训练**

我们采用了相似大小的数据块来训练模型，这主要是出于简化和效率的考虑。然而，在实际操作中，使用不同长度的输入来训练 LLM 也是有益的，这有助于模型在使用时能更好地适应各种类型的输入。

为了实现图 5.9 中展示的数据分割和加载，我们首先定义了一个 train_ratio，将 90% 的数据用于训练，其余的 10% 作为模型训练过程中的验证数据：

In [37]:
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

使用 train_data 和 val_data 子集，我们现在可以创建相应的数据加载器，复用第二章中的 `create_dataloader_v1` 代码：

In [40]:
from chapter02 import create_dataloader_v1
torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True
    )
val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False
    )

在上述代码中，我们选择了较小的批量大小，以降低计算资源的消耗，因为我们处理的是一个非常小的数据集。然而，在实际操作中，使用 1024 或更大的批量大小来训练 LLM 是常见的做法。

作为一项可选的检查，我们可以遍历数据加载器，以确保它们是正确创建的：

In [41]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)
    
print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


我们可以看到如下输出：

```
Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])
```

根据前述代码的运行结果，我们共得到了 9 个训练批次，每个批次包含 2 个样本，每个样本有 256 个 token。由于我们仅为验证过程分配了 10% 的数据，因此只有一个验证批次，其中包含 2 个输入示例。

正如我们预期的那样，输入数据（x）和目标数据（y）具有相同的形状（批次大小乘以每个批次中的 token 数量），因为目标就是输入数据向后移动一个位置，这一点我们在第二章中已经讨论过。

接下来，我们将实现一个实用函数，用于计算训练和验证加载器返回的特定批次的交叉熵损失：

In [42]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device) #A
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(
    logits.flatten(0, 1), target_batch.flatten()
    )
    return loss

现在，我们可以使用这个用于计算单个批次损失的实用函数` calc_loss_batch`，来实现以下函数`calc_loss_loader`，该函数计算给定数据加载器采样的所有批次的损失：

**代码列表 5.2 计算训练和验证损失的函数**

In [43]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if num_batches is None:
        num_batches = len(data_loader) #A
    else:
        num_batches = min(num_batches, len(data_loader)) #B
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item() #C
        else:
            break
    return total_loss / num_batches #D

默认情况下，`calc_loss_batch` 函数会遍历给定数据加载器中的所有批次，将各批次的损失累积在 `total_loss` 变量中，然后计算并平均所有批次的损失。另外，我们也可以通过 `num_batches` 参数来指定较少的批次数量，以便在模型训练过程中加快评估速度。

现在，我们将这个 `calc_loss_batch` 函数应用于训练集和验证集的加载器，看看它在实际操作中的表现：

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #A
model.to(device)
train_loss = calc_loss_loader(train_loader, model, device) #B
val_loss = calc_loss_loader(val_loader, model, device)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

产生的损失值如下：

```
Training loss: 10.98758347829183
Validation loss: 10.98110580444336
```

由于模型尚未经过训练，因此损失值相对较高。作为对比，如果模型能够学习到如何按照训练集和验证集中的顺序生成下一个 token，那么损失值将会接近于 0。

现在我们已经掌握了一种衡量生成文本质量的方法，接下来，我们将训练 LLM 以减少这种损失，从而使其在生成文本方面表现得更好，如图 5.10 所示。

**图 5.10 我们已经回顾了文本生成过程，并实现了基本的模型评估技术以计算训练集和验证集的损失。接下来，我们将进入了解训练函数，并对 LLM 进行预训练。**

![fig5.10](https://github.com/Pr04Ark/llms-from-scratch-cn/blob/trans01/Translated_Book/img/fig-5-10.jpg?raw=true)

如图 5.10 所示，接下来的部分将重点放在 LLM 的预训练上。模型训练完成后，我们将采用不同的文本生成策略，并保存及加载预训练的模型权重。